In [ ]:
import os
from PIL import Image, ImageDraw, ImageFont
import arabic_reshaper
from bidi.algorithm import get_display

In [ ]:
def get_wrapped_text(text, font, max_width):
    words = text.split()
    lines = []
    current_line = []
    current_width = 0

    for word in words:
        word_width = font.getbbox(word)[2] - font.getbbox(word)[0]
        space_width = font.getbbox(' ')[2] - font.getbbox(' ')[0]
        
        if current_width + word_width + space_width <= max_width:
            current_line.append(word)
            current_width += word_width + space_width
        else:
            if current_line:
                lines.append(' '.join(current_line))
            current_line = [word]
            current_width = word_width + space_width

    if current_line:
        lines.append(' '.join(current_line))

    return lines

In [ ]:
def text_to_image(text, font_path, font_size, output_path, max_width=1200, line_spacing=10):
    try:
        # Reshape the text
        reshaped_text = arabic_reshaper.reshape(text)
        
        font = ImageFont.truetype(font_path, font_size)

        # Get wrapped lines before applying bidi algorithm
        wrapped_lines = get_wrapped_text(reshaped_text, font, max_width)
        
        # Apply bidi algorithm to each line separately
        bidi_lines = [get_display(line) for line in wrapped_lines]

        max_line_width = max(font.getbbox(line)[2] - font.getbbox(line)[0] for line in bidi_lines)
        total_height = len(bidi_lines) * (font_size + line_spacing) - line_spacing

        image = Image.new('RGB', (max_width + 40, total_height + 40), color='white')
        draw = ImageDraw.Draw(image)

        y_position = 20
        for line in bidi_lines:
            bbox = font.getbbox(line)
            x_position = max_width + 20 - bbox[2]  # Align to the right
            draw.text((x_position, y_position), line, font=font, fill='black')
            y_position += font_size + line_spacing

        image.save(output_path)
        # print(f"Image saved successfully: {output_path}")
    except Exception as e:
        print(f"Error in text_to_image function: {str(e)}")



In [ ]:
def process_txt_files(root_dir, output_dir, font_path, font_size):
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.endswith('.txt'):
                file_path = os.path.join(dirpath, filename)
                try:
                    with open(file_path, 'r', encoding='utf-8') as file:
                        content = file.read()
                        
                        # Create relative path for output
                        rel_path = os.path.relpath(dirpath, root_dir)
                        
                        # Create output directories
                        images_output_subdir = os.path.join(output_dir, rel_path)
                        texts_output_subdir = os.path.join(output_dir,'text-files' , rel_path)
                        
                        # Create directories if they don't exist
                        os.makedirs(images_output_subdir, exist_ok=True)
                        os.makedirs(texts_output_subdir, exist_ok=True)
                        
                        # Create output paths
                        image_filename = os.path.splitext(filename)[0] + '.png'
                        image_path = os.path.join(images_output_subdir, image_filename)
                        text_path = os.path.join(texts_output_subdir, filename)
                        
                        # Create image from text content
                        text_to_image(content, font_path, font_size, image_path)
                        #print(f"Image saved: {image_path}")
                        
                        # Check if text file already exists
                        if not os.path.exists(text_path):
                            # Save text file
                            with open(text_path, 'w', encoding='utf-8') as text_file:
                                text_file.write(content)
                            #print(f"Text saved: {text_path}")
                        else:
                            print(f"Text file already exists: {text_path}")
                        
                except Exception as e:
                    print(f"Error processing {file_path}: {str(e)}")

In [ ]:
# text files
root_dir = "C:/Users/QAZ/Downloads/Compressed/news-dataset/prj"
output_path_t = "D:imagessss/"

In [ ]:
font_path_m = "C:/Users/QAZ/AppData/Local/Microsoft/Windows/Fonts/"
font_size = [16,18,20,22,24]
font_names = ["Nazanin.ttf", "BArash.ttf", "BArshia.ttf", "BDavat.ttf", "SBARDIYA.TTF", "Fantezy.ttf", "Jaleh_MRT.ttf", "Smooth Light_MRT.ttf", "Ghalam-2_MRT.TTF", "NasimB.ttf"]

In [ ]:
for font in font_names:
    font_path = font_path_m + font
    #print(font_path)
    for size in font_size:
        output_path = output_path_t + font + '-' + str(size)
        process_txt_files(root_dir, output_path, font_path, size)